<a href="https://colab.research.google.com/github/FinancialEngineerLab/SSVILibrary/blob/main/SSVI_local_vol_temp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from scipy.stats import norm
from scipy import random
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
from mpl_toolkits.mplot3d import axes3d
from matplotlib.colors import cnames
import pandas as pd
from scipy.optimize import bisect

In [ ]:
# arbitrage-free #

# phi function declare! #
def phi(theta, params):
    gamma, sigma, rho = params
    return 1./(gamma*theta)*(1.0-(1.0-np.exp(-gamma*theta))/(gamma*theta))

### SVI = w function###

def SSVI(x, t, params):
    gamma, sigma, rho = params
    theta = sigma*sigma *t
    p = phi(theta, params)
    return 0.5*theta*(1.0 + rho*p*x + np.sqrt((p*x+rho)*(p*x+rho)+1.-rho*rho))


def SSVIprime(x, t, params):
    ## First derivative with respect to x
    gamma, sigma, rho = params
    theta = sigma*sigma*t
    p = phi(theta, params)
    return 0.5*theta*p*(p*x+rho*np.sqrt(p*p*x*x+2.*p*rho*x+1.)+rho)/np.sqrt(p*p*x*x+2.*p*rho*x+1.)

def SSVIdoublePrime(x, t, params):
    ## Second derivative with respect to x
    gamma, sigma, rho = params
    theta = sigma*sigma*t
    p = phi(theta, params)
    return 0.5*theta*p*p*(1.-rho*rho)/ ((p*p*x*x + 2.*p*rho*x + 1.)*np.sqrt(p*p*x*x+2.*p*rho*x+1.))

def SSVITImeDerivative(x, t ,params):
    gamma, sigma, rho = params
    eps = 0.0001
    return (SSVI(x, t+eps, params)- SSVI(x,t-eps, params)) / (2.0*eps)

def g(x,t, params):
    w = SSVI(x,t,params)
    w1= SSVIprime(x,t,params)
    w2 = SSVIdoublePrime(x, t, params)
    return (1.0 - 0.5*x*w1/w) * (1.0 - 0.5*x*w1/w) - 0.25*w1*w1*(0.25+1.0/w)

def dminus(x, t, params):
    vSqrt = np.sqrt(SSVI(x,t,params))
    return -x/Vsqrt - 0.5*vSqrt

def densitySSVI(x, t, params):
    dm = dminus(x,t,params)
    return g(x,t,params)*np.exp(-0.5*dm*dm)/np.sqrt(2.0*np.pi*SSVI(x,t,params))

def SSVI_LocalVarg(x,t,params):
    return SSVITImeDerivative(x,t,params)/g(x,t,params)


In [ ]:
## Numerical Example : SSVI Local Vol ##
# no calibration #
sigma, gamma, rho = 0.2, 0.8, -0.7
xx, TT = np.linspace(-1.0, 1.0, 50), np.linspace(0.001, 5.0, 50)

print('Consistency Check for Arbitrage Free (static:',(gamma - 0.25*(1.+np.abs(rho))>0.))
params = gamma, sigma, rho
localVarianceSSVI = [[SSVI_LocalVarg(x, t, params) for x in xx] for t in TT]
impliedVarianceSSVI = [[SSVI(x, t, params) for x in xx] for t in TT]


In [ ]:
# visualization #
fig = plt.figure(figsize = (8,5))
ax = fig.gca(projection='3d')
xxx, TTT = np.meshgrid(xx,TT)
localVarianceSSVIarr = np.array(localVarianceSSVI)
impliedVarianceSSVIarr = np.array(impliedVarianceSSVI)
ax.plot_surface(xxx, TTT, localVarianceSSVIarr, cmap=plt.cm.jet)
ax.set_xlabel("Log - Moneyness")
ax.set_ylabel("Mat")
ax.set_zlabel("Local Var")
ax.set_title("SSVI Local Vol Surface")
plt.show()

In [ ]:
### MC Pricing & BSM ###
def MC_localVol_pricing(s0, strike, T, params, nbSimul, nbSteps):
    deltaT = T / (1.*nbSteps)
    SS =np.linspace(s0, s0, nbSimul)
    time = np.linspace(deltaT, T, nbSteps)
    
    for t in time:
        sig = np.sqrt(SSVI_LocalVarg(np.log(SS), t, params))
        SS = SS*np.exp(-0.5*sig*sig*deltaT + np.sqrt(deltaT)*sig*random.normal(0., 1., nbSimul))
    price = np.mean(np.maximum(SS - strike, 0.))
    return price

In [ ]:
def Gdensity(x): ## Gaussian density
    return np.exp(-x*x/2.)/np.sqrt(2*np.pi)

#### Black Sholes Vega
def BlackScholesVegaCore(DF,F,X,T,v):
    vsqrt=v*np.sqrt(T)
    d1 = (np.log(F/X)+(vsqrt*vsqrt/2.))/vsqrt
    return F*Gdensity(d1)*np.sqrt(T)/DF

#### Black Sholes Function
def BlackScholesCore(CallPutFlag,DF,F,X,T,v):
    ## DF: discount factor
    ## F: Forward
    ## X: strike
    vsqrt=v*np.sqrt(T)
    d1 = (np.log(F/X)+(vsqrt*vsqrt/2.))/vsqrt
    d2 = d1-vsqrt
    if CallPutFlag:
        return DF*(F*norm.cdf(d1)-X*norm.cdf(d2))
    else:
        return DF*(X*norm.cdf(-d2)-F*norm.cdf(-d1))
    
##  Black-Scholes Pricing Function
def BlackScholes(CallPutFlag,S,X,T,r,d,v):
    ## r, d: continuous interest rate and dividend
    return BlackScholesCore(CallPutFlag,np.exp(-r*T),np.exp((r-d)*T)*S,X,T,v)

def BSimpliedvol(r,T,s0,K, price):
    ## Bisection algorithm when the Lee-Li algorithm breaks down
    def smileMin(vol, *args):
        K, s0, T, r, price = args
        return price - BlackScholes(True, s0, K, T, r, 0., vol)
    vMin, vMax = 0.000001, 2.
    
    return bisect(smileMin, vMin, vMax, args=(K, s0, T, r, price), xtol=1e-20, rtol=1e-15, full_output=False, disp=True)

In [ ]:
## Comparison of Implied Vol ##
sigma, gamma, rho = 0.2, 0.8, -0.7
s0, x, T = 1., 0., 1.
nbSimul = 10000
nbSteps = 100
xx = np.linspace(-0.5, 0.5, 10)
params = gamma, sigma, rho
sigma, rho

print("Consistency check for arbitrage - free: ", (gamma - 0.25*(1.+np.abs(rho))>0.))

ssvivol = np.sqrt(SSVI(x, T, params)/T)
BSPrice = BlackScholesCore(True,1.,s0,s0*np.exp(x),T,ssvivol)
print("Black-Scholes implied vol: ", BSimpliedvol(0., T, s0, s0*np.exp(x), BSPrice))
print("SSVI implied vol: ", ssvivol)

price = MC_localVol_pricing(s0, s0*np.exp(x), T, params, nbSimul, nbSteps)
print("Implied vol from local volatility price by MC: ",  BSimpliedvol(0., T, s0, s0*np.exp(x), price))

In [ ]:
### Another Phi Function ###
def phi(theta, params):
    gamma, eta, sigma, rho = params
    return eta / pow(theta, gamma)

def SSVI(x, t, params):
    gamma, eta, sigma, rho = params
    theta = sigma*sigma*t
    p = phi(theta, params)
    return 0.5*theta*(1.+rho*p*x+np.sqrt((p*x+rho)*(p*x+rho)+1.-rho*rho))

def SSVIprime(x, t, params):
    ## First derivative with respect to x
    gamma, eta, sigma, rho = params
    theta = sigma*sigma*t
    p = phi(theta, params)
    return 0.5*theta*p*(p*x+rho*np.sqrt(p*p*x*x+2.*p*rho*x+1.)+rho)/np.sqrt(p*p*x*x+2.*p*rho*x+1.)

def SSVIdoublePrime(x, t, params):
    ## Second derivative with respect to x
    gamma, eta, sigma, rho = params
    theta = sigma*sigma*t
    p = phi(theta, params)
    return 0.5*theta*p*p*(1.-rho*rho)/ ((p*p*x*x + 2.*p*rho*x + 1.)*np.sqrt(p*p*x*x+2.*p*rho*x+1.))
    
def SSVItimeDerivative(x, t, params):
    ## First derivative with respect to t, by central difference
    eps = 0.0001
    return (SSVI(x,t+eps,params) - SSVI(x,t-eps,params))/(2.*eps)

def g(x, t, params):
    w = SSVI(x,t,params)
    w1 = SSVIprime(x,t,params)
    w2 = SSVIdoublePrime(x,t,params)
    return (1.-0.5*x*w1/w)*(1.-0.5*x*w1/w) - 0.25*w1*w1*(0.25 + 1./w) + 0.5*w2
    

def dminus(x, t, params):
    vsqrt = np.sqrt(SSVI(x,t,params))
    return -x/vsqrt - 0.5*vsqrt
    
def densitySSVI(x, t, params):
    dm = dminus(x,t,params)
    return g(x,t,params)*np.exp(-0.5*dm*dm)/np.sqrt(2.*np.pi*SSVI(x,t,params))

def SSVI_LocalVarg(x, t, params):
    ## Compute the equivalent SSVI local variance
    return SSVItimeDerivative(x,t,params) / g(x,t,params)

In [ ]:
sigma, gamma, eta, rho = 0.2, 0.4, 0.2, -0.4
xx, TT = np.linspace(-1., 1.,20), np.linspace(0.1, 2., 20)


print("Consistency check to avoid static arbitrage: ", (gamma - 0.25*(1.+np.abs(rho))>0.))
params = gamma, eta, sigma, rho

localVarianceSSVI_2 = [[SSVI_LocalVarg(x, t, params) for x in xx] for t in TT]

In [ ]:
fig = plt.figure(figsize=(8,5))
ax = fig.gca(projection='3d')
xxx, TTT = np.meshgrid(xx, TT)
localVarianceSSVI_2arr = np.array(localVarianceSSVI_2)
ax.plot_surface(xxx, TTT, localVarianceSSVI_2arr, cmap=plt.cm.jet, rstride=1, cstride=1, linewidth=0)
ax.set_xlabel("Log-moneyness")
ax.set_ylabel("Maturity")
ax.set_zlabel("Local variance")
ax.set_title("SSVI local variance")
plt.show()